In [ ]:
# This R environment comes with many helpful analytics packages installed
# It is defined by the kaggle/rstats Docker image: https://github.com/kaggle/docker-rstats
# For example, here's a helpful package to load

library(tidyverse) # metapackage of all tidyverse packages

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

list.files(path = "../input")

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Project Data Analysis for Finance: Performa Cabang
## Latar Belakang
DQLab Finance merupakan perusahaan finance yang sudah mempunyai banyak cabang tersebar dimana-mana. Sejak berdiri pada Januari 2020, 
DQLab Finance konsisten menyalurkan pembiayaan untuk masyarakat dan semakin berkembang setiap bulannya dengan membuka cabang baru.

Walaupun berumur kurang dari 1 tahun, DQLab Finance sudah mempunyai banyak cabang, oleh karena itu perlu dipantau bagaimana performa dari cabang - cabang tersebut.

Pada masing-masing cabang, terdapat agen-agen yang bertugas mencari dan mendata calon mitra yang akan mengajukan pinjaman kepada DQLab Finance. 
Lalu jika sudah disetujui, agen juga yang akan memberikan uang tersebut kepada mitra.

## Tugas dan langkah
**Tugas :**
Menganalisis Performa cabang pada bulan Mei 2020.

**Langkah -langkah :**
1. Memfilter data untuk bulan Mei 2020
2. Membuat summary per cabang untuk melihat data 5 cabang terbaik dan terburuk
3. Karena cabang bertambah setiap bulannya, maka perlu dicek umur cabang dan performa mei
4. Mencari cabang terburuk untuk masing - masing kelompok umur

## Load Dataset

In [ ]:
# Menyimpan hasil pada dataframe df_loan
# Menggunakan fungsi read.csv() untuk membaca file csv
# stringsAsFactors ini berguna agar data-data yang bertipe character tidak diubah menjadi factor
df_loan <- read.csv('https://dqlab-dataset.s3-ap-southeast-1.amazonaws.com/loan_disbursement.csv', stringsAsFactors = F)
# fungsi glimpse dari package dplyr(package yang paling sering digunakan dalam analisis data) untuk melihat data
# karena hanya pakai 1 fungsi, kita bisa memanggilnya tanpa load package
dplyr::glimpse(df_loan)

Terlihat bahwa ada 9,754 baris data (Observations) dan ada 5 kolom (Variables),
* loan_id : unik ID dari data ini
* tanggal_cair : tanggal uang diberikan kepada mitra
* cabang : lokasi agen bekerja dan tempat mitra terdaftar
* agen : petugas lapangan yang melakukan pencairan
* amount : jumlah uang yang dicairkan

In [ ]:
df_loan

# Summary data bulan lalu (Mei 2020)
## Memfilter data bulan Mei 2020, dan jumlahkan data per cabang

In [ ]:
# Gunakan pipe %>% untuk menyambungkan fungsi.
# Package yang digunakan dplyr sering digunakan dalam analisis data, sangat membantu dalam manipulasi data
library(dplyr)
# Menyimpan hasil pada dataframe df_loan_mei
df_loan_mei <- df_loan %>% 
  # Memfilter data dengan tanggal cair lebih besar sama dengan 1 mei dan lebih kecil sama dengan 31 mei
  filter(tanggal_cair >= '2020-05-01', tanggal_cair <= '2020-05-31') %>%
  # Kelompokkan data berdasarkan cabang
  group_by(cabang) %>%
  # Fungsi summarise() untuk mengubah beberapa nilai menjadi satu ringkasan nilai
  # Menghitung total amount dengan menjumlahkan variabel amount
  summarise(total_amount = sum(amount))
# Tampilkan hasil 
df_loan_mei

## Tampilkan data 5 cabang dengan total amount paling besar

In [ ]:
library(dplyr)
# Package Scales igunakan untuk memformat value data numerik menjadi format yang mudah dibaca
library(scales)
# Menggunakan dataframe df_loan_mei dari hasil sebelumnya
df_loan_mei %>%
  # Fungsi arrange() untuk mengurutkan baris data default secara ascending(menurun)
  # Mengurutkan data total_amount dari terbesar
  arrange(desc(total_amount)) %>% 
  # Fungsi mutate() untuk  membuat variabel baru berdasarkan variabel yang ada
  # Fungsi comma() mengubah numerik menjadi ada simbol ribuan
  mutate(total_amount = comma(total_amount)) %>%
  # Menampilkan 5 data teratas
  head(5)

## Tampilkan data 5 cabang dengan total amount paling kecil

In [ ]:
library(dplyr)
library(scales)
df_loan_mei %>% 
  arrange(total_amount) %>% 
  mutate(total_amount = comma(total_amount)) %>% 
  head(5)

**Kesimpulan :**
Terjadi perbedaan yang sangat signifikan antara top 5 dengan bottom 5. Hal ini mungkin karena umur cabang yang berbeda beda karena ada pertumbuhan cabang baru setiap bulannya.

# Melihat hubungan umur cabang dengan total amount
## Menghitung umur cabang (dalam bulan)
Karena tidak tersedia data umur cabang, maka perlu dihitung terlebih dahulu,
yakni dengan menghitung sudah berapa lama sejak tanggal cair pertama sampai dengan bulan Mei.

In [ ]:
library(dplyr)
# Menyimpan hasil pada dataframe df_cabang_umur
df_cabang_umur <- df_loan %>%
  group_by(cabang) %>%
  # Mencari tanggal pertama_cair berdasarkan nilai terkecil tanggal_cair
  summarise(pertama_cair = min(tanggal_cair)) %>% 
  # Untuk memudahkan cara perhitungan umur dengan membagi jumlah selisih hari dengan 30, 
  # karena itu tanggal batas nya menggunakan tanggal tengah bulan (2020-05-15), 
  # agar tidak terlalu mempengaruhi presisi perhitungan.
  mutate(umur = as.numeric(as.Date('2020-05-15') - as.Date(pertama_cair)) %/% 30) 
df_cabang_umur

## Gabungkan data umur dan performa mei

In [ ]:
library(dplyr)
# Menyimpan hasil pada dataframe df_loan_mei_umur
df_loan_mei_umur <- df_cabang_umur %>%
  # Menggabungkan df_cabang_umur dengan df_loan_mei menggunakan fungsi inner_join()
  inner_join(df_loan_mei, by = 'cabang')
df_loan_mei_umur

## Plot relasi umur dan performa mei

In [ ]:
# Package yang digunakan untuk membuat plot dengan syntax yang konsisten
library(ggplot2)

ggplot(df_loan_mei_umur, aes(x = umur, y = total_amount)) +
  geom_point() +
  scale_y_continuous(labels = scales::comma) +
  labs(title = "Semakin berumur, performa cabang akan semakin baik",
       x = "Umur(bulan)",
       y = "Total Amount")

**Kesimpulan :**
Kesimpulan
Terlihat bahwa ada pola semakin tua cabang, maka performa nya semakin baik.
Hal ini karena cabang tersebut masih berkembang sehingga belum sampai pada performa maksimal.

Akan tetapi pada masing - masing umur itu juga ada cabang yang performanya dibawah yang lain.

# Cabang dengan performa rendah pada kelompok umur
## Mencari cabang yang perfoma rendah untuk setiap umur

In [ ]:
library(dplyr)
library(scales)
# Menyimpan hasil pada dataframe df_loan_mei_flag menggunakan data pada df_loan_mei_umur
df_loan_mei_flag <- df_loan_mei_umur %>% 
  # Kelompokkan berdasarkan umur
  group_by(umur) %>% 
  # Hitung nilai Quartile dan Inter Quartile Range
  mutate(Q1 = quantile(total_amount, 0.25),
	   Q3 = quantile(total_amount, 0.75),
	   IQR = (Q3-Q1)) %>%
  # Buat flag dengan ketentuan rendah untuk total_amout lebih kecil dari (Q1 - IQR)
  mutate(flag = ifelse(total_amount < (Q1 - IQR), 'rendah','baik'))
# Tampilkan hasil hanya flag rendah
df_loan_mei_flag %>% 
  filter(flag == 'rendah') %>% 
  mutate_if(is.numeric, funs(comma))

## Buat Scatterplot lagi dan beri warna merah pada cabang yang rendah 

In [ ]:
library(ggplot2)

ggplot(df_loan_mei_flag, aes(x = umur, y = total_amount)) +
  geom_point(aes(color = flag)) +
  # Mengatur legend pada grafik
  scale_color_manual(breaks = c("baik", "rendah"),
                        values=c("blue", "red")) +
  scale_y_continuous(labels = scales::comma) +
  labs(title = "Ada cabang berpeforma rendah padahal tidak termasuk bottom 5 nasional",
	   color = "",
       x = "Umur(bulan)",
       y = "Total Amount")

# Analisis cabang dengan performa rendah
## Lihat perbadingan performa cabang di umur yang sama
Selanjutnya akan dianalisis lebih lanjut kenapa cabang itu bisa performanya rendah di bulan mei

Untuk kali ini akan dilihat hanya untuk yang umur 3 bulan saja, dilihat detail performa pada bulan mei dengan mengihitung,
- jumlah hari pencairan dalam 1 bulan,
- jumlah agen yang aktif,
- total loan yang cair,
- rata - rata amount cair per loan.

In [ ]:
library(dplyr)
library(scales)
df_loan_mei_flag %>% 
  filter(umur == 3) %>% 
  inner_join(df_loan, by = 'cabang') %>% 
  filter(tanggal_cair >= '2020-05-01', tanggal_cair <= '2020-05-31') %>% 
  group_by(cabang, flag) %>% 
  summarise(jumlah_hari = n_distinct(tanggal_cair),
            agen_aktif = n_distinct(agen),
            total_loan_cair = n_distinct(loan_id),
            avg_amount = mean(amount), 
            total_amount = sum(amount)) %>% 
  arrange(total_amount) %>% 
  mutate_if(is.numeric, funs(comma))

## Lihat perbadingan performa agen pada cabang yang rendah
Dari hasil eksplorasi sebelumnya, terlihat bahwa yang berbeda jauh hanya total_loan_cair saja.
Jumlah hari dan jumlah agen dalam 1 bulan sama semua.

Selanjutnya perlu dilihat bagaimana perbandingan nya per agent.

Untuk melanjutkan, dilihat untuk yang umur 3 bulan dan flag nya rendah dilihat detail performa pada bulan mei per agen dengan mengihitung,

- jumlah hari pencairan dalam 1 bulan,
- total loan yang cair,
- rata - rata amount cair per loan
- total amount cair

In [ ]:
library(dplyr)
library(scales)
df_loan_mei_flag %>% 
  filter(umur == 3, flag == 'rendah') %>% 
  inner_join(df_loan, by = 'cabang') %>% 
  filter(tanggal_cair >= '2020-05-01', tanggal_cair <= '2020-05-31') %>% 
  group_by(cabang, agen) %>% 
  summarise(jumlah_hari = n_distinct(tanggal_cair),
            total_loan_cair = n_distinct(loan_id),
            avg_amount = mean(amount), 
            total_amount = sum(amount)) %>% 
  arrange(total_amount) %>% 
  mutate_if(is.numeric, funs(comma))

## Lihat perbadingan performa agen pada cabang yang paling baik umur 3 bulan
Pada tabel sebelumnya, terlihat pula bahwa ada cabang yang punya 3 agen, tapi performa nya jauh diatas cabang AE, bahkan yang paling tinggil diantara cabang lain pada umur tersebut, lebih tinggi dari yang mempunya 4 agen cabang tersebut adalah cabang AH.

Dengan cara yang hampir sama, akan dilihat bagaimana performa masing-masing agen dari cabang AH tersebut. Hanya saja untuk ini bisa langsung pakai data df_loan lalu filter nama cabang nya saja.

In [ ]:
library(dplyr)
library(scales)
df_loan %>% 
  filter(cabang == 'AH') %>% 
  filter(tanggal_cair >= '2020-05-01', tanggal_cair <= '2020-05-31') %>% 
  group_by(cabang, agen) %>% 
  summarise(jumlah_hari = n_distinct(tanggal_cair),
            total_loan_cair = n_distinct(loan_id),
            avg_amount = mean(amount), 
            total_amount = sum(amount)) %>% 
  arrange(total_amount) %>% 
  mutate_if(is.numeric, funs(comma))

**Kesimpulan :**
Berdasarkan analisis tersebut, dapat disimpulkan bahwa rendahnya performa dari cabang AE adalah karena salah satu agen yang melakukan pencairan hanya 4 hari dalam 1 bulan, padahal agen lain bisa aktif 21 hari.

Hal ini membuat total amount dari agen tersebut hanya 20% dibandingkan agen yang lainnya.

Sedangkan pada cabang AH, performanya sangat baik karena ketiga agen melakukan pencairan hampir / selalu setiap hari kerja. 2 orang full 21 hari 1 orang 19 hari. Sehingga performa nya terjaga dengan baik.

Perlu diperhatikan juga bahwa untuk membandingkan performa cabang itu sebaiknya di kelompokkan dulu berdasarkan karakteristik yang sama. Tidak langsung semua cabang dibandingkan tanpa mengetahui karakteristik nya.

Apalagi ketika dalam real world nanti cabang ini bisa terletak di berbeda wilayah (pulau misalnya) yang mempunyai kultur yang berbeda

Selanjutnya perlu dianalisis lebih lanjut kenapa ada agen yang hanya aktif beberapa hari saja dalam sebulan.

Untuk kedepannya setiap agen agar dipastikan untuk bisa aktif setiap hari nya, sehingga bisa menjaga performa cabang.

# Source : https://academy.dqlab.id/main/project